In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [5]:
#Lectura asignacion Credifiel
sheet_cred2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for j in sheet_cred2:
    asignacion_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Asignacion/'+j+'')
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
asignacion_cred = asignacion_cred.loc[:,['CREDITO','ARANCEL']]
#asignacion_cred2 = pd.pivot_table(asignacion_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
#asignacion_cred2 = pd.DataFrame(asignacion_cred2.to_records())

In [6]:
#Asignamos valores a los parametros \n"
#today = date.today().strftime('%Y%m%d')
today = 20201231
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [7]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [8]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [9]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Llamadas detallado Credifiel

In [10]:
#Hacemos la consulta referente a las gestiones de Credifiel
llamadas_cred = consulta_sql('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^CRED)')
llamadas_cred.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_cred.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1606803067.8041239,Inbound,2020-12-01 00:11:08,1301_IN,CREDIFIEL_CALL,999,Default inbound list,26748421,5529935415,...,0 days 00:11:08,0,00:00 - 01:00,ANSWER,16,200 OK,INBOUND,552,CDMX,CENTRO


In [11]:
llamadas_cred['turno'] = np.where(llamadas_cred['call_time_hour']<=14,'M','V')

In [12]:
#Obtenemos el performance bruto de cada gestor
control_gestores_bruto_cred = pd.pivot_table(llamadas_cred,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_bruto_cred = to_dataframe(control_gestores_bruto_cred)
control_gestores_bruto_cred = control_gestores_bruto_cred.iloc[:,[0,1,4,2,3]]
control_gestores_bruto_cred.columns = ['Gestor','Turno','NumeroGestionesB','TiempoTotalSegB','PromedioLlamadaSegB']
control_gestores_bruto_cred['TiempoTotalMinB'] = [x/60 for x in control_gestores_bruto_cred['TiempoTotalSegB']]
control_gestores_bruto_cred

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB
0,ALEJANDRA YESENIA HERNANDEZ BITE,V,15,592,39.466667,9.866667
1,GARCIA CHAVEZ BRAYAN,M,1509,134254,88.968854,2237.566667
2,GARCIA CHAVEZ BRAYAN,V,539,33406,61.977737,556.766667
3,GONZALEZ MORALES BRENDA MARLEN,M,611,28393,46.469722,473.216667
4,GONZALEZ MORALES BRENDA MARLEN,V,107,4470,41.775701,74.500000
5,HERNANDEZ GALINDO RODRIGO,M,576,25578,44.406250,426.300000
6,HERNANDEZ VELASCO YALITZI AZAYAEEL,M,6,267,44.500000,4.450000
7,Inbound No Agent,M,218,11782,54.045872,196.366667
8,Inbound No Agent,V,184,10341,56.201087,172.350000
9,LAZARO LOPEZ VICTOR MANUEL,M,78,3165,40.576923,52.750000


In [13]:
#Obtenemos el performance de las llamadas contestadas
llamadas_cred2 = llamadas_cred.loc[llamadas_cred['status_name'].isin(['Contacto con titular','Promesa de pago','Mensaje con terceros','Mensaje con familiar','Negativa de pago','Contacto familiar','Agendar llamada'])]
control_gestores_llamadas_cred = pd.pivot_table(llamadas_cred2,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_llamadas_cred = to_dataframe(control_gestores_llamadas_cred)
control_gestores_llamadas_cred = control_gestores_llamadas_cred.iloc[:,[0,1,4,2,3]]
control_gestores_llamadas_cred.columns = ['Gestor','Turno','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_llamadas_cred['TiempoTotalMin'] = [x/60 for x in control_gestores_llamadas_cred['TiempoTotalSeg']]
control_gestores_llamadas_cred

,Gestor,Turno,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,ALEJANDRA YESENIA HERNANDEZ BITE,V,1,8,8.000000,0.133333
1,GARCIA CHAVEZ BRAYAN,M,29,4788,165.103448,79.800000
2,GARCIA CHAVEZ BRAYAN,V,10,1129,112.900000,18.816667
3,GONZALEZ MORALES BRENDA MARLEN,M,24,2008,83.666667,33.466667
4,GONZALEZ MORALES BRENDA MARLEN,V,1,74,74.000000,1.233333
5,HERNANDEZ GALINDO RODRIGO,M,45,5812,129.155556,96.866667
6,LAZARO LOPEZ VICTOR MANUEL,M,2,659,329.500000,10.983333
7,LAZARO LOPEZ VICTOR MANUEL,V,45,6646,147.688889,110.766667
8,MARTINEZ RODRIGUEZ JUAN,V,1,83,83.000000,1.383333
9,MEDINA ARIAS MAURICIO,M,20,3583,179.150000,59.716667


In [14]:
control_cred_turno = pd.merge(control_gestores_bruto_cred,control_gestores_llamadas_cred,how='left',on=['Gestor','Turno'])
control_cred_turno.fillna(0,inplace=True)
control_cred_turno.head(2)

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,ALEJANDRA YESENIA HERNANDEZ BITE,V,15,592,39.466667,9.866667,1.0,8.0,8.000000,0.133333
1,GARCIA CHAVEZ BRAYAN,M,1509,134254,88.968854,2237.566667,29.0,4788.0,165.103448,79.800000


In [15]:
#Obtenemos el performance de las llamadas contestadas
control_gestores_total_cred = pd.pivot_table(llamadas_cred2,index=['full_name'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_total_cred = to_dataframe(control_gestores_total_cred)
control_gestores_total_cred = control_gestores_total_cred.iloc[:,[0,3,1,2]]
control_gestores_total_cred.columns = ['Gestor','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_total_cred['TiempoTotalMin'] = [x/60 for x in control_gestores_total_cred['TiempoTotalSeg']]
control_gestores_total_cred

,Gestor,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,ALEJANDRA YESENIA HERNANDEZ BITE,1,8,8.000000,0.133333
1,GARCIA CHAVEZ BRAYAN,39,5917,151.717949,98.616667
2,GONZALEZ MORALES BRENDA MARLEN,25,2082,83.280000,34.700000
3,HERNANDEZ GALINDO RODRIGO,45,5812,129.155556,96.866667
4,LAZARO LOPEZ VICTOR MANUEL,47,7305,155.425532,121.750000
5,MARTINEZ RODRIGUEZ JUAN,1,83,83.000000,1.383333
6,MEDINA ARIAS MAURICIO,20,3583,179.150000,59.716667
7,MISHELLLE ADYANI CHAVEZ CASTRO,82,14063,171.500000,234.383333
8,QUINTOS ARVIZU LUIS DANIEL,83,9973,120.156627,166.216667
9,ZARZA NORIA JEFREY ALBERTO,10,620,62.000000,10.333333


# Performance Credifiel

In [16]:
#Hacemos la consulta referente al performance del Gestor
performance_all = consulta_sql('procesos_externos.all_rpt_agent_performance',mes,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [17]:
performance_cred = performance_all.loc[performance_all['user_group']=='AFINES']
performance_cred[['ocupacion','adherencia']] = percent_tonumeric(performance_cred,['ocupacion','adherencia'])

In [18]:
performance_cred['hora_login'] = [int(x[:2]) for x in performance_cred['login_vici']]
performance_cred['hora_logout'] = [int(x[:2]) for x in performance_cred['logout_vici']]
performance_cred['turno'] = np.where((performance_cred['hora_logout']<=14)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'M',
                                    np.where((performance_cred['hora_logout']>17)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'V','Mixto'))

In [19]:
performance_cred['horas_trabajadas'] = performance_cred['hora_logout']-performance_cred['hora_login']
performance_cred['llamadas_hora'] = performance_cred['llamadas']/performance_cred['horas_trabajadas']
performance_cred['llamadas_hora'] = performance_cred['llamadas_hora'].round(1)
performance_cred.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
89,2020-12-01,GACHAVEZBRA,GARCIA CHAVEZ BRAYAN,AFINES,136,00:01:18,33.23,64.35,07:09:39,14:00:06,...,00:00:00,00:00:00,00:00:33,00:51:56,00:00:01,7,14,M,7,19.4


In [20]:
control_performance_cred_turno = pd.pivot_table(performance_cred,index=['nombre_usuario','turno'],values=['llamadas','ocupacion','llamadas_hora'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred_turno = to_dataframe(control_performance_cred_turno)
control_performance_cred_turno = control_performance_cred_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
control_performance_cred_turno.columns = ['Gestor','Turno','SumLlamadasDia','SumLlamadasHora','SumOcupacion','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion','DiasAsitencia','MinLlamadasDia','MinLlamadasHora','MinOcupacion','MaxLlamadasDia','MaxLlamadasHora','MaxOcupacion']
control_performance_cred_turno.head(1)

,Gestor,Turno,SumLlamadasDia,SumLlamadasHora,SumOcupacion,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion,DiasAsitencia,MinLlamadasDia,MinLlamadasHora,MinOcupacion,MaxLlamadasDia,MaxLlamadasHora,MaxOcupacion
0,AGUILAR RIOS DANIEL ALEJANDRO,M,3077,509.3,360.3,341.888889,56.588889,40.033333,9,108,44.4,29.16,487,81.2,58.45


In [21]:
control_performance_cred_turno2 = control_performance_cred_turno[['Gestor','Turno','PromedioLlamadasHora','PromedioOcupacion']]

In [22]:
control_performance_cred = pd.pivot_table(performance_cred,index=['nombre_usuario'],values=['llamadas','ocupacion','llamadas_hora','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred = to_dataframe(control_performance_cred)
control_performance_cred = control_performance_cred.iloc[:,[0,1,2,6,7,8]]
control_performance_cred.columns = ['Gestor','SumHorasTrabajadas','SumLlamadasDia','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion']
control_performance_cred.head(3)

,Gestor,SumHorasTrabajadas,SumLlamadasDia,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion
0,AGUILAR RIOS DANIEL ALEJANDRO,68,3857,350.636364,56.981818,38.271818
1,CASTILLEJO CASTILLEJO DULCE MICHELLE,21,655,218.333333,31.200000,50.483333
2,GARCIA CHAVEZ BRAYAN,209,2748,119.478261,13.460870,27.853478


In [23]:
conjunto_performance_llamadas_cred = pd.merge(control_gestores_total_cred,control_performance_cred,on='Gestor',how='left')
conjunto_performance_llamadas_cred['ContactoHora'] = conjunto_performance_llamadas_cred['NumeroGestiones']/conjunto_performance_llamadas_cred['SumHorasTrabajadas']
conjunto_performance_llamadas_cred = conjunto_performance_llamadas_cred.iloc[:,[0,8,10,9,]]
conjunto_performance_llamadas_cred = conjunto_performance_llamadas_cred.dropna()
conjunto_performance_llamadas_cred

,Gestor,PromedioLlamadasHora,ContactoHora,PromedioOcupacion
1,GARCIA CHAVEZ BRAYAN,13.460870,0.186603,27.853478
2,GONZALEZ MORALES BRENDA MARLEN,23.320000,0.806452,31.842000
3,HERNANDEZ GALINDO RODRIGO,32.300000,2.500000,34.006667
4,LAZARO LOPEZ VICTOR MANUEL,19.950000,1.270270,45.940000
5,MARTINEZ RODRIGUEZ JUAN,80.000000,1.000000,49.560000
6,MEDINA ARIAS MAURICIO,23.871429,0.136054,39.541429
7,MISHELLLE ADYANI CHAVEZ CASTRO,24.707692,0.820000,29.401538
8,QUINTOS ARVIZU LUIS DANIEL,39.029167,0.340164,37.922083
9,ZARZA NORIA JEFREY ALBERTO,inf,0.149254,29.334286


# Gestion detallado Credifiel

In [24]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = consulta_sql('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
6,9229469,1,2020-12-01,0 days 07:29:39,CR-267152,LIEVANO CAL Y MAYOR VERONICA,9616189250,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,4294.00,9616189250 TT REALIZARA PAGO 6 $4294 15/12/20,PDP,Promesa de Pago,CONTACTO
7,9229522,1,2020-12-01,0 days 07:34:46,CR-494364,DE LA VEGA ROMERO MIGUEL ANGEL,5959250197,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,3148.83,5959250197 SE GENERA PAGO 9 POR $3148.83 PARA ...,PDP,Promesa de Pago,CONTACTO
11,9229759,1,2020-12-01,0 days 07:49:21,CR-503789,MARTINEZ MANRIQUE LUCY,5588471578,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-01,2000.00,5588471578 TT REALIZARA LIQUIDACION CON 32000...,PDP,Promesa de Pago,CONTACTO


In [25]:
#Hacemos la consulta de la asignacion
asignacion_cred2 = consulta_sql('credifiel_rpt_asignacion_activa')
asignacion_cred2 = asignacion_cred2.iloc[:,[1,3,32]]
asignacion_cred2.columns = ['unegocio_id','credito','division']
#asignacion_brad2['credito'] = pd.to_numeric(asignacion_brad2['credito'])
asignacion_cred2.head(3)

,unegocio_id,credito,division
0,1,CR-101922,COBRANZA EXTRAJUDICIAL
1,1,CR-101949,LIQUIDAR
2,1,CR-101952,LIQUIDAR


In [26]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred2,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['division'].fillna('COBRANZA',inplace=True)
consolidado_cred['division'].replace('','COBRANZA',inplace=True)
consolidado_cred['credito'] = [x[3:] for x in consolidado_cred['credito']]
consolidado_cred['credito'] = pd.to_numeric(consolidado_cred['credito'])
consolidado_cred.head(3)

,folio_gestion,unegocio_id_x,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,unegocio_id_y,division
0,9229469,1,2020-12-01,0 days 07:29:39,267152,LIEVANO CAL Y MAYOR VERONICA,9616189250,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,4294.00,9616189250 TT REALIZARA PAGO 6 $4294 15/12/20,PDP,Promesa de Pago,CONTACTO,1.0,EXTRAJUDICIAL
1,9229522,1,2020-12-01,0 days 07:34:46,494364,DE LA VEGA ROMERO MIGUEL ANGEL,5959250197,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,3148.83,5959250197 SE GENERA PAGO 9 POR $3148.83 PARA ...,PDP,Promesa de Pago,CONTACTO,1.0,COBRANZA EXTRAJUDICIAL
2,9229759,1,2020-12-01,0 days 07:49:21,503789,MARTINEZ MANRIQUE LUCY,5588471578,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-01,2000.00,5588471578 TT REALIZARA LIQUIDACION CON 32000...,PDP,Promesa de Pago,CONTACTO,NaN,COBRANZA


In [27]:
pagos_cred_fact = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
pagos_cred_fact['ARANCEL'] = np.where(pagos_cred_fact['ARANCEL'].isna(),np.mean(pagos_cred_fact['ARANCEL']),pagos_cred_fact['ARANCEL'])

In [28]:
consolidado_pagos_cred = pd.merge(consolidado_cred,pagos_cred_fact,how='left',left_on=['credito'],right_on=['ID'])
try: consolidado_pagos_cred['fecha_promesa'] = pd.to_datetime(consolidado_pagos_cred['fecha_promesa'],dayfirst=False)
except: consolidado_pagos_cred['fecha_promesa'] = consolidado_pagos_cred['fecha_gestion']

In [29]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
#consolidado_pagos_cred['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(4, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_cred['fecha_promesa'],consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['FECHA'])]
#consolidado_pagos_cred['PagoUnico'] = [x.item() for x in consolidado_pagos_cred['PagoUnico']]

In [30]:
consolidado_pagos_cred['PagoReal'] = [x*y for x,y in zip(consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['ARANCEL'])]
consolidado_pagos_cred2 = consolidado_pagos_cred.drop_duplicates(subset=['credito','IMPORTE','FECHA','usuario'])

In [31]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred2,index=['nombre_usuario'],values=['monto_promesa','IMPORTE','PagoReal'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,3,1,6,4,5]]
pagos_totales_cred.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
pagos_totales_cred.fillna(0,inplace=True)
pagos_totales_cred['EfVolumen'] = pagos_totales_cred['NumeroPagos']/pagos_totales_cred['NumeroPromesas']
pagos_totales_cred['EfPagos'] = pagos_totales_cred['SumaPagos']/pagos_totales_cred['SumaPromesas']
#pagos_totales_cred['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_cred.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel,EfVolumen,EfPagos
0,GARCIA CHAVEZ BRAYAN,47,35,170290.98,113946.82,21200.9054,0.744681,0.66913


In [32]:
gestor_final_cred = pagos_totales_cred[['Gestor','SumaPagos','EfPagos']]

In [33]:
final_cred = pd.merge(gestor_final_cred,conjunto_performance_llamadas_cred,how='outer',on='Gestor')
final_cred['PromedioLlamadasHora'] = final_cred['PromedioLlamadasHora'].round()
final_cred['PromedioOcupacion'] = final_cred['PromedioOcupacion']/100
cols = ['PromedioOcupacion','EfPagos','ContactoHora']
final_cred[cols] = final_cred[cols].round(2)
final_cred.fillna(0,inplace=True)
final_cred

,Gestor,SumaPagos,EfPagos,PromedioLlamadasHora,ContactoHora,PromedioOcupacion
0,GARCIA CHAVEZ BRAYAN,113946.82,0.67,13.0,0.19,0.28
1,GONZALEZ MORALES BRENDA MARLEN,41862.51,1.17,23.0,0.81,0.32
2,HERNANDEZ GALINDO RODRIGO,49417.35,1.76,32.0,2.50,0.34
3,LAZARO LOPEZ VICTOR MANUEL,4050.00,0.85,20.0,1.27,0.46
4,MEDINA ARIAS MAURICIO,30355.88,0.94,24.0,0.14,0.40
5,MISHELLLE ADYANI CHAVEZ CASTRO,2300.00,0.08,25.0,0.82,0.29
6,QUINTOS ARVIZU LUIS DANIEL,71958.39,0.75,39.0,0.34,0.38
7,ZARZA NORIA JEFREY ALBERTO,44495.50,0.63,inf,0.15,0.29
8,MARTINEZ RODRIGUEZ JUAN,0.00,0.00,80.0,1.00,0.50


In [34]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/Credifiel/Reporte Gestor Credifiel '+now+'.xlsx',engine='xlsxwriter')
final_cred.to_excel(writer,'BSC',index=False,header=True)
gestor_final_cred.to_excel(writer,'MontoRecuperado',index=False,header=True)
control_cred_turno.to_excel(writer,'LlamadasDetallado',index=False,header=True)
control_performance_cred_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
pagos_totales_cred.to_excel(writer,'PagosDetallado',index=False,header=True)

writer.save()